In [4]:
# Install OpenAI client and configure ARK connection
!pip install openai ipywidgets

from openai import OpenAI

# We will use Ark's OpenAI compabible endpoint, allowing us to use the
# standard OpenAI SDK.
ARK_ENDPOINT = "http://ark-api.default.127.0.0.1.nip.io:8080/openai/v1"

# Create OpenAI client. For local development, an API key is not needed.
# For prod environments, you'll need to create/request an API key.
client = OpenAI(
    base_url=ARK_ENDPOINT,
    api_key="notused",
)

print("Configuration complete. Run the cells below to interact with ARK.")

Configuration complete. Run the cells below to interact with ARK.


In [5]:
# List available targets and select one
# ARK lists all available query targets (models, agents, teams, tools) as "models"
# so that they can be queried using the standard OpenAI endpoints
import ipywidgets as widgets
from IPython.display import display, clear_output

targets = client.models.list()
target_names = [target.id for target in targets.data]

print("Available targets:")
for target in target_names:
    print(f"  - {target}")

# Create dropdown for target selection
target_dropdown = widgets.Dropdown(
    options=target_names,
    value=target_names[0] if target_names else None,
    description='Select Target:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Create text area for message input
message_textarea = widgets.Textarea(
    value='Hello! How are you?',
    placeholder='Enter your message here...',
    description='Message:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px', height='100px')
)

# Create radio buttons for streaming mode
streaming_radio = widgets.RadioButtons(
    options=[('Non-streaming', False), ('Streaming', True)],
    value=False,
    description='Response Mode:',
    style={'description_width': 'initial'}
)

# Create execute button
execute_button = widgets.Button(
    description='Execute',
    button_style='success',
    layout=widgets.Layout(width='100px')
)

# Create output area
output_area = widgets.Output()

def on_execute_click(b):
    with output_area:
        clear_output()
        print(f"Executing request to {target_dropdown.value}...")
        print(f"Message: {message_textarea.value}")
        print("-" * 50)
        
        try:
            if streaming_radio.value:
                # Streaming response
                stream = client.chat.completions.create(
                    model=target_dropdown.value,
                    messages=[{"role": "user", "content": message_textarea.value}],
                    stream=True
                )
                print("Response:")
                for chunk in stream:
                    if chunk.choices and len(chunk.choices) > 0:
                        if chunk.choices[0].delta and chunk.choices[0].delta.content is not None:
                            print(chunk.choices[0].delta.content, end="")
                print()
            else:
                # Non-streaming response
                response = client.chat.completions.create(
                    model=target_dropdown.value,
                    messages=[{"role": "user", "content": message_textarea.value}],
                    stream=False
                )
                print("Response:")
                if response.choices and len(response.choices) > 0:
                    print(response.choices[0].message.content)
                else:
                    print("No response content received")
        except Exception as e:
            print(f"Error: {e}")

execute_button.on_click(on_execute_click)

print("\nSelect a target, enter your message, choose response mode, and click Execute:")
display(target_dropdown)
display(message_textarea)
display(streaming_radio)
display(execute_button)
display(output_area)

Available targets:
  - agent/ark-pentester
  - agent/aws-operator-agent
  - agent/code-reviewer
  - agent/deep-research
  - agent/github-agent
  - agent/github-dev-assistant
  - agent/golang-weather-agent
  - agent/idea-generation
  - agent/json-output-agent
  - agent/noah
  - agent/planner
  - agent/query-with-parameter-reference
  - agent/sample-agent
  - agent/team-leader
  - team/coding-team
  - team/testing-team
  - model/claude-3-5-haiku
  - model/claude-3-5-sonnet
  - model/claude-4-opus
  - model/claude-haiku-bedrock
  - model/default
  - model/gemini-1-5-flash
  - model/gemini-2-5-flash-lite
  - model/test1
  - tool/ark-mcp-list-agents
  - tool/ark-mcp-query-agent
  - tool/get-coordinates
  - tool/github-add-comment-to-pending-review
  - tool/github-add-issue-comment
  - tool/github-add-sub-issue
  - tool/github-assign-copilot-to-issue
  - tool/github-cancel-workflow-run
  - tool/github-create-and-submit-pull-request-review
  - tool/github-create-branch
  - tool/github-create-

Dropdown(description='Select Target:', layout=Layout(width='400px'), options=('agent/ark-pentester', 'agent/aw…

Textarea(value='Hello! How are you?', description='Message:', layout=Layout(height='100px', width='600px'), pl…

RadioButtons(description='Response Mode:', options=(('Non-streaming', False), ('Streaming', True)), style=Desc…

Button(button_style='success', description='Execute', layout=Layout(width='100px'), style=ButtonStyle())

Output()